# 🚀 Deploy Qwen QwQ 32B Reasoning Model on Amazon SageMaker AI with Auto Scale Down To Zero

## Introduction: [Qwen QwQ 32B](https://huggingface.co/Qwen/QwQ-32B)

[QwQ](https://huggingface.co/Qwen/QwQ-32B) is the reasoning model of the Qwen series. Compared with conventional instruction-tuned models, QwQ, which is capable of thinking and reasoning, can achieve significantly enhanced performance in downstream tasks, especially hard problems. QwQ-32B is the medium-sized reasoning model, which is capable of achieving competitive performance against state-of-the-art reasoning models, e.g., DeepSeek-R1, o1-mini.

- **Mathematical Reasoning**: Achieves an impressive 90.6% on MATH-500, outperforming both Claude 3.5 (78.3%) and matching OpenAI's o1-mini (90.0%)
- **Advanced Mathematics**: Scores 50.0% on AIME (American Invitational Mathematics Examination), significantly 'higher than Claude 3.5 (16.0%)
- **Scientific Reasoning**: Demonstrates strong performance on GPQA with 65.2%, on par with Claude 3.5 (65.0%)
- **Programming**: Achieves 50.0% on LiveCodeBench, showing competitive performance with leading proprietary models

> [NOTE]
> QwQ-32B is released under the Apache 2.0 license, making it suitable for both research and commercial applications.

Let's get started deploying one of the most capable open-source reasoning models available today!

In [ ]:
%pip install -Uq sagemaker boto3 --force-reinstall --no-cache-dir --quiet --no-warn-conflicts

In [ ]:
import json
import sagemaker
import boto3
import sys
import time
from typing import List, Dict
from datetime import datetime
from sagemaker.huggingface import (
    HuggingFaceModel, 
    get_huggingface_llm_image_uri
)

boto_region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session(boto_session=boto3.Session(region_name=boto_region))
role = sagemaker.get_execution_role()
sagemaker_client = boto3.client("sagemaker")
sagemaker_runtime_client = boto3.client("sagemaker-runtime")
s3_client = boto3.client("s3")

prefix = sagemaker.utils.unique_name_from_base("DEMO")
print(f"prefix: {prefix}")

## Setup your SageMaker Real-time Endpoint 
### Create a SageMaker endpoint configuration

We begin by creating the endpoint configuration and set MinInstanceCount to 0. This allows the endpoint to scale in all the way down to zero instances when not in use.

There are a few parameters we want to setup for our endpoint. We first start by setting the variant name, and instance type we want our endpoint to use. In addition we set the *model_data_download_timeout_in_seconds* and *container_startup_health_check_timeout_in_seconds* to have some guardrails for when we deploy inference components to our endpoint. In addition we will use Managed Instance Scaling which allows SageMaker to scale the number of instances based on the requirements of the scaling of your inference components. We set a *MinInstanceCount* and *MinInstanceCount* variable to size this according to the workload you want to service and also maintain controls around cost. Lastly, we set *RoutingStrategy* for the endpoint to optimally tune how to route requests to instances and inference components for the best performance.

The suggested instance types to host the QwQ 32B model can be `ml.g5.12xlarge`, `ml.g6.12xlarge`, `ml.g6e.12xlarge`.

In [ ]:
# Set an unique endpoint config name
endpoint_config_name = f"{prefix}-endpoint-config"
print(f"Demo endpoint config name: {endpoint_config_name}")

# Set varient name and instance type for hosting
variant_name = "AllTraffic"
instance_type = "ml.g5.12xlarge"
model_data_download_timeout_in_seconds = 3600
container_startup_health_check_timeout_in_seconds = 3600

min_instance_count = 0 # Minimum instance must be set to 0
max_instance_count = 3

sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ExecutionRoleArn=role,
    ProductionVariants=[
        {
            "VariantName": variant_name,
            "InstanceType": instance_type,
            "InitialInstanceCount": 1,
            "ModelDataDownloadTimeoutInSeconds": model_data_download_timeout_in_seconds,
            "ContainerStartupHealthCheckTimeoutInSeconds": container_startup_health_check_timeout_in_seconds,
            "ManagedInstanceScaling": {
                "Status": "ENABLED",
                "MinInstanceCount": min_instance_count,
                "MaxInstanceCount": max_instance_count,
            },
            "RoutingConfig": {"RoutingStrategy": "LEAST_OUTSTANDING_REQUESTS"},
        }
    ],
)

### Create the SageMaker endpoint
Next, we create our endpoint using the above endpoint config

In [ ]:
# Set a unique endpoint name
endpoint_name = f"{prefix}-endpoint"
print(f"Demo endpoint name: {endpoint_name}")

sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name,
)

In [ ]:
sagemaker_session.wait_for_endpoint(endpoint_name)

## Deploy using HuggingFace TGI Container
Hugging Face Large Language Model (LLM) Inference Deep Learning Container (DLC) on Amazon SageMaker enables developers to efficiently deploy and serve open-source LLMs at scale. This DLC is powered by Text Generation Inference (TGI), an open-source, purpose-built solution optimized for high-performance text generation tasks. 

**Key Features of HuggingFace TGI Containers:**

* **Tensor Parallelism**: Distributes computation across multiple GPUs, allowing the deployment of large models that exceed the memory capacity of a single GPU.
* **Dynamic Batching**: Aggregates multiple incoming requests into a single batch, enhancing throughput and resource utilization.
* **Optimized Transformers Code**: Utilizes advanced techniques like flash-attention to improve inference speed and efficiency for popular model architectures like DeepSeek, Llama, Falcon, Mistal, Mixtral and many more.

**Benefits for Deploying LLMs with HuggingFace TGI on Amazon SageMaker:**

* **Simplified Deployment**: TGI containers provide a low-code interface, allowing users to specify configurations like model parallelization and optimization settings through straightforward configuration files. 
* **Performance Optimization**: By leveraging optimized inference libraries and techniques, such as tensor parallelism and dynamic batching, these containers enhance inference performance, reducing latency and improving throughput. 
* **Scalability**: Designed to handle large models, TGI containers enable efficient scaling across multiple GPUs or specialized hardware like AWS Inferentia, ensuring that even the most demanding models can be deployed effectively.

For a more exhaustive list, please refer to this [TGI Release Page](https://github.com/aws/deep-learning-containers/releases?q=tgi+AND+gpu&expanded=true)

### Create Model Artifact
We will be deploying the Qwen 32B model using the TGI container. In order to do so you need to set the image you would like to use with the proper configuartion. You can also create a SageMaker model to be referenced when you create your inference component


In [ ]:
tgi_inference_image_uri = get_huggingface_llm_image_uri(
     "huggingface", 
     version="2.3.1"
)
print(f"Using TGI Image: {tgi_inference_image_uri}")
qwen_qwq_32b = "Qwen/QwQ-32B"
qwen_tgi_model = {
    "Image": tgi_inference_image_uri,
    "Environment": {
        "HF_MODEL_ID": qwen_qwq_32b,
        "ENDPOINT_SERVER_TIMEOUT": "3600",
        "MESSAGES_API_ENABLED": "true",
        "SAGEMAKER_MODEL_SERVER_WORKERS": "1",
        "SM_NUM_GPUS": "4",
        "MAX_TOTAL_TOKENS": "8192",
        "MAX_INPUT_TOKENS": "4096",
        'HF_HUB_ENABLE_HF_TRANSFER': "1",
        "PORT": "8080"
    },
}
model_name_tgi = f"qwen-qwq-32b-tgi-{datetime.now().strftime('%y%m%d-%H%M%S')}"
# create SageMaker Model
sagemaker_client.create_model(
    ModelName=model_name_tgi,
    ExecutionRoleArn=role,
    Containers=[qwen_tgi_model],
)

We can now create the Inference Components which will deployed on the endpoint that you specify. Please note here that you can provide a SageMaker model or a container to specification. If you provide a container, you will need to provide an image and artifactURL as parameters. In this example we set it to the model name we prepared in the cells above. You can also set the `ComputeResourceRequirements` to supply SageMaker what should be reserved for each copy of the inference component. You can also set the copy count of the number of Inference Components you would like to deploy. These can be managed and scaled as the capabilities become available. 

Note that in this example we set the `NumberOfAcceleratorDevicesRequired` to a value of `4`. By doing so we reserve 4 accelerators for each copy of this inference component so that we can use tensor parallel. 

In [ ]:
inference_component_name_qwen = f"{prefix}-IC-qwen-32b-{datetime.now().strftime('%y%m%d-%H%M%S')}"
variant_name = "AllTraffic"

sagemaker_client.create_inference_component(
    InferenceComponentName=inference_component_name_qwen,
    EndpointName=endpoint_name,
    VariantName=variant_name,
    Specification={
        "ModelName": model_name_tgi,
        "ComputeResourceRequirements": {
            "NumberOfAcceleratorDevicesRequired": 4,
            "NumberOfCpuCoresRequired": 1,
            "MinMemoryRequiredInMb": 1024,
        },
    },
    RuntimeConfig={"CopyCount": 1},
)

Wait until the inference component is `InService`.

In [ ]:
import time
# Let's see how much it takes
start_time = time.time()
while True:
    desc = sagemaker_client.describe_inference_component(
        InferenceComponentName=inference_component_name_qwen
    )
    status = desc["InferenceComponentStatus"]
    print(status)
    sys.stdout.flush()
    if status in ["InService", "Failed"]:
        break
    time.sleep(30)
total_time = time.time() - start_time
print(f"\nTotal time taken: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")

#### Invoke endpoint with boto3
Note that you can also invoke the endpoint with boto3. If you have an existing endpoint, you don't need to recreate the `predictor` and can follow below example to invoke the endpoint with an endpoint name.

In [ ]:
import boto3
import json
sagemaker_runtime = boto3.client('sagemaker-runtime')

prompt = {
    'messages':[
    {"role": "user", "content": "How many R are in STRAWBERRY? Keep your answer and explanation short!"}
],
    'temperature':0,
    'max_tokens':512,
}
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    InferenceComponentName=inference_component_name_qwen,
    ContentType="application/json",
    Body=json.dumps(prompt)
)
response_dict = json.loads(response['Body'].read().decode("utf-8"))
response_content = response_dict['choices'][0]['message']['content']
print(response_content)

#### Streaming response from the endpoint
Additionally, SGLang allows you to invoke the endpoint and receive streaming response. Below is an example of how to interact with the endpoint with streaming response.

In [ ]:
import io
import json

# Example class that processes an inference stream:
class SmrInferenceStream:
    
    def __init__(self, sagemaker_runtime, endpoint_name, inference_component_name=None):
        self.sagemaker_runtime = sagemaker_runtime
        self.endpoint_name = endpoint_name
        self.inference_component_name = inference_component_name
        # A buffered I/O stream to combine the payload parts:
        self.buff = io.BytesIO() 
        self.read_pos = 0
        
    def stream_inference(self, request_body):
        # Gets a streaming inference response 
        # from the specified model endpoint:
        response = self.sagemaker_runtime\
            .invoke_endpoint_with_response_stream(
                EndpointName=self.endpoint_name, 
                InferenceComponentName=self.inference_component_name,
                Body=json.dumps(request_body), 
                ContentType="application/json"
        )
        # Gets the EventStream object returned by the SDK:
        event_stream = response['Body']
        for event in event_stream:
            # Passes the contents of each payload part
            # to be concatenated:
            self._write(event['PayloadPart']['Bytes'])
            # Iterates over lines to parse whole JSON objects:
            for line in self._readlines():
                line = line.decode('utf-8')[len('data: '):]
                # print(line)
                try:
                    resp = json.loads(line)
                except:
                    continue
                if len(line)>0 and type(resp) == dict:
                    # if len(resp.get('choices')) == 0:
                    #     continue
                    part = resp.get('choices')[0]['delta']['content']
                    
                else:
                    part = resp
                # Returns parts incrementally:
                yield part
    
    # Writes to the buffer to concatenate the contents of the parts:
    def _write(self, content):
        self.buff.seek(0, io.SEEK_END)
        self.buff.write(content)

    # The JSON objects in buffer end with '\n'.
    # This method reads lines to yield a series of JSON objects:
    def _readlines(self):
        self.buff.seek(self.read_pos)
        for line in self.buff.readlines():
            self.read_pos += len(line)
            yield line[:-1]

In [ ]:
request_body = {
    'messages':[
        {"role": "user", "content": "How many R are in STRAWBERRY? Keep your answer and explanation short!"},
    ],
    'temperature':0,
    'max_tokens':512,
    'stream': True,
    'stream_options': {'include_usage': True}
}

smr_inference_stream = SmrInferenceStream(
    sagemaker_runtime, endpoint_name, inference_component_name_qwen)
stream = smr_inference_stream.stream_inference(request_body)
for part in stream:
    print(part, end='')

## Automatically Scale To Zero
### Scaling policies
Once the endpoint is deployed and InService, you can then add the necessary scaling policies:

* A [target tracking](https://docs.aws.amazon.com/autoscaling/application/userguide/application-auto-scaling-target-tracking.html) policy that can scale in the copy count for our inference component model copies to zero, and from 1 to n. 
* A [step scaling policy](https://docs.aws.amazon.com/autoscaling/application/userguide/application-auto-scaling-step-scaling-policies.html) policy that will allow the endpoint to scale out from zero.

These policies work together to provide cost-effective scaling - the endpoint can scale to zero when idle and automatically scale out as needed to handle incoming requests.

### Scaling policy for inference components copies (target tracking)
We start with creating our target tracking policies for scaling the CopyCount of our inference component

#### Register a new autoscaling target
After you create your SageMaker endpoint and inference components, you register a new auto scaling target for Application Auto Scaling. In the following code block, you set **MinCapacity**  to **0**, which is required for your endpoint to scale down to zero

In [ ]:
aas_client = sagemaker_session.boto_session.client("application-autoscaling")
cloudwatch_client = sagemaker_session.boto_session.client("cloudwatch")

# Autoscaling parameters
resource_id = f"inference-component/{inference_component_name_qwen}"
service_namespace = "sagemaker"
scalable_dimension = "sagemaker:inference-component:DesiredCopyCount"

min_copy_count = 0
max_copy_count = 3

aas_client.register_scalable_target(
    ServiceNamespace=service_namespace,
    ResourceId=resource_id,
    ScalableDimension=scalable_dimension,
    MinCapacity=min_copy_count,
    MaxCapacity=max_copy_count,
)

#### Configure Target Tracking Scaling Policy
Once you have registered your new scalable target, the next step is to define your target tracking policy. In the code example that follows, we set the TargetValue to 5. This setting instructs the auto-scaling system to increase capacity when the number of concurrent requests per model reaches or exceeds 5. Here we are taking advantage of the more granular auto scaling metric `PredefinedMetricType`: `SageMakerInferenceComponentConcurrentRequestsPerCopyHighResolution` to more accurately monitor and react to changes in inference traffic. Take a look this [blog](https://aws.amazon.com/blogs/machine-learning/amazon-sagemaker-inference-launches-faster-auto-scaling-for-generative-ai-models/) for more information. 

In [ ]:
aas_client.describe_scalable_targets(
    ServiceNamespace=service_namespace,
    ResourceIds=[resource_id],
    ScalableDimension=scalable_dimension,
)

# The policy name for the target traking policy
target_tracking_policy_name = f"Target-tracking-policy-qwen-qwq-scale-to-zero-aas-{inference_component_name_qwen}"

aas_client.put_scaling_policy(
    PolicyName=target_tracking_policy_name,
    PolicyType="TargetTrackingScaling",
    ServiceNamespace=service_namespace,
    ResourceId=resource_id,
    ScalableDimension=scalable_dimension,
    TargetTrackingScalingPolicyConfiguration={
        "PredefinedMetricSpecification": {
            "PredefinedMetricType": "SageMakerInferenceComponentConcurrentRequestsPerCopyHighResolution",
        },
        # Low TPS + load TPS
        "TargetValue": 5,  # you need to adjust this value based on your use case
        "ScaleInCooldown": 300,  # default
        "ScaleOutCooldown": 300,  # default
    },
)

Application Auto Scaling creates two CloudWatch alarms per scaling target. The first triggers scale-out actions after 30 seconds (using 3 sub-minute data point), while the second triggers scale-in after 15 minutes (using 90 sub-minute data points). The time to trigger the scaling action is usually 1–2 minutes longer than those minutes because it takes time for the endpoint to publish metrics to CloudWatch, and it also takes time for AutoScaling to react. 

### Scale out from zero policy (step scaling policy )
To enable your endpoint to scale out from zero instances, do the following:

#### Configure Step Scaling Policy
Create a step scaling policy that defines when and how to scale out from zero. This policy will add 1 model copy when triggered, enabling SageMaker to provision the instances required to handle incoming requests after being idle.  The following shows you how to define a step scaling policy. Here we have configured to scale out from 0 to 1 model copy ("ScalingAdjustment": 1), depending on your use case you can adjust ScalingAdjustment as required. 

In [ ]:
# The policy name for the step scaling policy
step_scaling_policy_name = f"Step-scaling-policy-qwen-qwq-scale-to-zero-aas-{inference_component_name_qwen}"

aas_client.put_scaling_policy(
    PolicyName=step_scaling_policy_name,
    PolicyType="StepScaling",
    ServiceNamespace=service_namespace,
    ResourceId=resource_id,
    ScalableDimension=scalable_dimension,
    StepScalingPolicyConfiguration={
        "AdjustmentType": "ChangeInCapacity",
        "MetricAggregationType": "Maximum",
        "Cooldown": 60,
        "StepAdjustments":
          [
             {
               "MetricIntervalLowerBound": 0,
               "ScalingAdjustment": 1
             }
          ]
    },
)

In [ ]:
resp = aas_client.describe_scaling_policies(
    PolicyNames=[step_scaling_policy_name],
    ServiceNamespace=service_namespace,
    ResourceId=resource_id,
    ScalableDimension=scalable_dimension,
)
step_scaling_policy_arn = resp['ScalingPolicies'][0]['PolicyARN']
print(f"step_scaling_policy_arn: {step_scaling_policy_arn}")

#### Create the CloudWatch alarm that will trigger our policy

Finally, create a CloudWatch alarm with the metric **NoCapacityInvocationFailures**. When triggered, the alarm initiates the previously defined scaling policy. For more information about the NoCapacityInvocationFailures metric, see [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/monitoring-cloudwatch.html#cloudwatch-metrics-inference-component).

We have also set the following:
- EvaluationPeriods to 1 
- DatapointsToAlarm to 1 
- ComparisonOperator to  GreaterThanOrEqualToThreshold

This results in 1 min waiting for the step scaling policy to trigger

In [ ]:
# The alarm name for the step scaling alarm
step_scaling_alarm_name = f"step-scaling-alarm-qwen-qwq-scale-to-zero-aas-{inference_component_name_qwen}"

cloudwatch_client.put_metric_alarm(
    AlarmName=step_scaling_alarm_name,
    AlarmActions=[step_scaling_policy_arn],  # Replace with your actual ARN
    MetricName='NoCapacityInvocationFailures',
    Namespace='AWS/SageMaker',
    Statistic='Maximum',
    Dimensions=[
        {
            'Name': 'InferenceComponentName',
            'Value': inference_component_name_qwen  # Replace with actual InferenceComponentName
        }
    ],
    Period=30, # Set a lower period 
    EvaluationPeriods=1,
    DatapointsToAlarm=1,
    Threshold=1,
    ComparisonOperator='GreaterThanOrEqualToThreshold',
    TreatMissingData='missing'
)

From cloudwatch console, you can check the alarms created.
![cloudwatch alarms](./img/cloudwatch-alarms.png)

### Testing the behaviour
Notice the `MinInstanceCount: 0` setting in the Endpoint configuration, which allows the endpoint to scale down to zero instances. With the scaling policy, CloudWatch alarm, and minimum instances set to zero, your SageMaker Inference Endpoint will now be able to automatically scale down to zero instances when not in use, helping you optimize your costs and resource utilization.

### IC copy count scales in to zero
We'll pause for a few minutes without making any invocations to our model. Based on our target tracking policy, when our SageMaker endpoint doesn't receive requests for about 10 to 15 minutes, it will automatically scale down to zero the number of model copies. 

In [ ]:
time.sleep(600)
start_time = time.time()
while True:
    desc = sagemaker_client.describe_inference_component(InferenceComponentName=inference_component_name_qwen)
    status = desc["InferenceComponentStatus"]
    print(status)
    sys.stdout.flush()
    if status in ["InService", "Failed"]:
        break
    time.sleep(30)

total_time = time.time() - start_time
print(f"\nTotal time taken: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")

desc = sagemaker_client.describe_inference_component(InferenceComponentName=inference_component_name_qwem)
print(desc)

### Endpoint's instances scale in to zero

After 10 additional minutes of inactivity, SageMaker automatically terminates all underlying instances of the endpoint, eliminating all associated costs.

In [ ]:
# after 10mins instances will scale down to 0
time.sleep(600)
# verify whether CurrentInstanceCount is zero
sagemaker_session.wait_for_endpoint(endpoint_name)

### Invoke the endpoint with a sample prompt

If we try to invoke our endpoint while instances are scaled down to zero, we get a validation error: `An error occurred (ValidationError) when calling the InvokeEndpoint operation: Inference Component has no capacity to process this request. ApplicationAutoScaling may be in-progress (if configured) or try to increase the capacity by invoking UpdateInferenceComponentRuntimeConfig API.`

In [ ]:
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    InferenceComponentName=inference_component_name_qwen,
    ContentType="application/json",
    Body=json.dumps(prompt)
)
response_dict = json.loads(response['Body'].read().decode("utf-8"))
response_content = response_dict['choices'][0]['message']['content']
print(response_content)

### Scale out from zero kicks in
However, after 1 minutes our step scaling policy should kick in. SageMaker will then start provisioning a new instance and deploy our inference component model copy to handle requests. This demonstrates the endpoint's ability to automatically scale out from zero when needed.

In [ ]:
start_time = time.time()
while True:
    desc = sagemaker_client.describe_inference_component(InferenceComponentName=inference_component_name_qwen)
    status = desc["InferenceComponentStatus"]
    print(status)
    sys.stdout.flush()
    if status in ["InService", "Failed"]:
        break
    time.sleep(30)

total_time = time.time() - start_time
print(f"\nTotal time taken: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")

desc = sagemaker_client.describe_inference_component(InferenceComponentName=inference_component_name_qwen)
print(desc)

#### verify that our endpoint has succesfully scaled out from zero

In [ ]:
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    InferenceComponentName=inference_component_name_qwen,
    ContentType="application/json",
    Body=json.dumps(prompt)
)
response_dict = json.loads(response['Body'].read().decode("utf-8"))
response_content = response_dict['choices'][0]['message']['content']
print(response_content)

## Cleanup

- Deregister scalable target
- Delete cloudwatch alarms
- Delete scaling policies
  
Make sure to delete the endpoint and other artifacts that were created to avoid unnecessary cost. You can also go to SageMaker AI console to delete all the resources created in this example.

In [ ]:
try:
    # Deregister the scalable target for AAS
    aas_client.deregister_scalable_target(
        ServiceNamespace="sagemaker",
        ResourceId=resource_id,
        ScalableDimension=scalable_dimension,
    )
    print(f"Scalable target for [b]{resource_id}[/b] deregistered. ✅")
except aas_client.exceptions.ObjectNotFoundException:
    print(f"Scalable target for [b]{resource_id}[/b] not found!.")

print("---" * 10)

# Delete CloudWatch alarms created for Step scaling policy
try:
    cloudwatch_client.delete_alarms(AlarmNames=[step_scaling_alarm_name])
    print(f"Deleted CloudWatch step scaling scale-out alarm [b]{step_scaling_alarm_name} ✅")
except cloudwatch_client.exceptions.ResourceNotFoundException:
    print(f"CloudWatch scale-out alarm [b]{step_scaling_alarm_name}[/b] not found.")


# Delete step scaling policies
print("---" * 10)

try:
    aas_client.delete_scaling_policy(
        PolicyName=step_scaling_policy_name,
        ServiceNamespace="sagemaker",
        ResourceId=resource_id,
        ScalableDimension="sagemaker:variant:DesiredInstanceCount",
    )
    print(f"Deleted scaling policy [i green]{step_scaling_policy_name} ✅")
except aas_client.exceptions.ObjectNotFoundException:
    print(f"Scaling policy [i]{step_scaling_policy_name}[/i] not found.")

In [ ]:
sagemaker_client.delete_inference_component(InferenceComponentName=inference_component_name_qwen)
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)